# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
Vacation_pd = pd.read_csv("Resources/Weather_Output.csv")
Vacation_pd


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Guerrero Negro,27.9769,-114.0611,65.77,65,55,11.43,MX,1605236400
1,1,Vaini,-21.2000,-175.2000,79.74,84,58,13.98,TO,1605236400
2,2,Birao,10.2849,22.7882,71.87,24,5,6.62,CF,1605236400
3,3,New Norfolk,-42.7826,147.0587,70.74,47,76,7.61,AU,1605236400
4,4,Inongo,-1.9500,18.2667,70.75,94,97,4.27,CD,1605236400
...,...,...,...,...,...,...,...,...,...,...
555,555,Hof,50.3167,11.9167,40.69,90,69,5.06,DE,1605236400
556,556,Kruisfontein,-34.0033,24.7314,61.18,88,0,5.93,ZA,1605236400
557,557,Kalat,29.0225,66.5916,51.71,47,15,4.88,PK,1605236400
558,558,Vestmanna,62.1564,-7.1664,48.61,91,100,19.69,FO,1605236400


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
gmaps.configure(api_key=g_key)
locations = Vacation_pd[["Lat", "Lng"]]
Humidity = Vacation_pd["Humidity"].astype(float)

In [29]:
gmaps.configure(api_key=g_key)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
Vacation_pd.isna
Vacation_clean_df=Vacation_pd.dropna()
Vacation_clean_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Guerrero Negro,27.9769,-114.0611,65.77,65,55,11.43,MX,1605236400
1,1,Vaini,-21.2000,-175.2000,79.74,84,58,13.98,TO,1605236400
2,2,Birao,10.2849,22.7882,71.87,24,5,6.62,CF,1605236400
3,3,New Norfolk,-42.7826,147.0587,70.74,47,76,7.61,AU,1605236400
4,4,Inongo,-1.9500,18.2667,70.75,94,97,4.27,CD,1605236400
...,...,...,...,...,...,...,...,...,...,...
555,555,Hof,50.3167,11.9167,40.69,90,69,5.06,DE,1605236400
556,556,Kruisfontein,-34.0033,24.7314,61.18,88,0,5.93,ZA,1605236400
557,557,Kalat,29.0225,66.5916,51.71,47,15,4.88,PK,1605236400
558,558,Vestmanna,62.1564,-7.1664,48.61,91,100,19.69,FO,1605236400


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
